In [ ]:
%pip install -r requirements.txt

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

## Step 1 - How to create a Neural Network
To  create a neural network in Pytorch, you need to create a class that extends the torch.nn.module

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class SineNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_layer = nn.Linear(1, 3)
        self.output_layer = nn.Linear(3, 1)

    def forward(self, x):
        hidden_output = self.hidden_layer(x)
        hidden_output = F.relu(hidden_output)
        output = self.output_layer(hidden_output)
        return output   

In [ ]:
epochs = 20

inputs = torch.randn(10000, 1)
true_values = torch.sin(inputs)


validation_inputs = torch.randn(1000, 1)
validation_true_values = torch.sin(validation_inputs)

model = SineNeuralNetwork()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epoch_losses = []
validation_losses = []

for epoch in trange(epochs):

    epoch_loss = 0
    for input, true_value in zip(inputs, true_values):
    

        output = model(input)
        loss = criterion(output, true_value)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    epoch_losses.append(epoch_loss/len(inputs))
    
    validation_loss = 0
    for input, true_value in zip(validation_inputs, validation_true_values):
        output = model(input)
        loss = criterion(output, true_value)
        validation_loss += loss.item()
    
    validation_losses.append(validation_loss/len(validation_inputs))

plt.plot(epoch_losses, label='Training loss')
plt.plot(validation_losses, label='Validation loss')
plt.legend()



In [ ]:
predictions = model(validation_inputs)
plt.scatter(validation_inputs, validation_true_values, label='True values')
plt.scatter(validation_inputs, predictions.detach().numpy(), label='Predictions')